# Visualizing the performance of Choco-Q

Reproduce the Table II in the paper "[Choco-Q: Commute Hamiltonian-based QAOA for Constrained Binary Optimization][1]" (HPCA 2025).

[1]: https://ieeexplore.ieee.org/document/TBD

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)  # display all rows
pd.set_option('display.max_columns', None)  # display all columns.

## Reproduce Table II

In [2]:
problem_scale = 4
file_path = f"./scale_{problem_scale}"

df1 = pd.read_csv(f"{file_path}/evaluate_depth.csv")
grouped_df1 = df1.groupby(['pkid', 'layers', 'method'], as_index=False).agg({
    "culled_depth": 'mean',
})
pivot_df1 = grouped_df1.pivot(index=['pkid'], columns='method', values=["culled_depth"])
method_order1 = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df1 = pivot_df1.reindex(columns=pd.MultiIndex.from_product([["culled_depth"], method_order1]))

df2 = pd.read_csv(f"{file_path}/evaluate_other.csv")
df2[['best_solution_probs', 'in_constraints_probs', 'iteration_count',
     'classcial', 'quantum', 'run_times']] = df2[['best_solution_probs', 'in_constraints_probs', 'iteration_count',
                                                  'classcial', 'quantum', 'run_times']].apply(pd.to_numeric, errors='coerce')
grouped_df2 = df2.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count': 'mean',
    'classcial': 'mean',
    'run_times': 'mean',
})
pivot_df2 = grouped_df2.pivot(index=['pkid', 'variables', 'constraints'], columns='method', values=["best_solution_probs", 'in_constraints_probs', 'ARG'])
method_order2 = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df2 = pivot_df2.reindex(columns=pd.MultiIndex.from_product([["best_solution_probs", 'in_constraints_probs', 'ARG'], method_order2]))

merged_df = pd.merge(pivot_df1, pivot_df2, on='pkid', how='inner')
merged_df = merged_df[['best_solution_probs', 'in_constraints_probs', 'ARG', 'culled_depth']]
merged_df = merged_df.rename(columns={
    'best_solution_probs': 'Success rate (%)',
    'in_constraints_probs': 'In-constraints rate (%)',
    'ARG': 'Approximation ratio gap (ARG)',
    'culled_depth': 'Circuit depth',
})
merged_df = merged_df.rename(columns={
    'PenaltySolver': 'Penalty',
    'CyclicSolver': 'Cyclic',
    'HeaSolver': 'HEA',
    'ChocoSolver': 'Choco-Q'
})
merged_df.index = [f"F{i}" for i in range(1, problem_scale + 1)] + [f"G{i}" for i in range(1, problem_scale + 1)] + [f"K{i}" for i in range(1, problem_scale + 1)]
merged_df

Success rate (%)                                 In-constraints rate (%)  \
            Penalty     Cyclic       HEA    Choco-Q                 Penalty   
F1         5.644531   7.177734  3.750000  44.619141               19.111328   
F2         0.029297   0.058594  0.009766  50.400391                0.439453   
F3         0.000000   0.000000  0.000000  25.566406                0.048828   
F4         0.000000   0.032552  0.000000  31.982422                0.000000   
G1         0.722656  13.896484  0.146484  26.142578                3.105469   
G2         0.078125   0.068359  0.000000  17.539062                0.371094   
G3         0.000000   0.000000  0.000000   0.009766                0.039062   
G4         0.000000   0.000000  0.000000   0.000000                0.000000   
K1         1.298828  10.166016  1.630859  44.687500                2.841797   
K2         0.000000   0.000000  0.000000  10.937500                0.019531   
K3         0.000000   0.000000  0.000000   0.000000                0.000000   
K4              NaN   0.000000  0.000000   2.060547                     NaN   

                                     Approximation ratio gap (ARG)  \
       Cyclic        HEA     Choco-Q                       Penalty   
F1  44.531250  12.832031  100.000000                  5.215580e+01   
F2   2.255859   0.273438  100.000000                  9.510269e+01   
F3   2.871094   0.000000  100.000000                  8.489098e+01   
F4   0.374349   0.000000  100.000000                  1.830051e+02   
G1  20.097656   0.732422  100.000000                  1.498747e+02   
G2  49.843750   0.087891  100.000000                  1.951689e+02   
G3  39.785156   0.000000  100.000000                  3.892068e+02   
G4  66.585286   0.000000   99.980469                  3.671064e+02   
K1  32.773438   5.312500  100.000000                  1.672865e+10   
K2  60.068359   0.009766  100.000000                  6.402153e+02   
K3  50.048828   0.000000  100.000000                  8.435748e+02   
K4  66.634115   0.000000  100.000000                           NaN   

                                             Circuit depth                \
          Cyclic           HEA       Choco-Q       Penalty Cyclic    HEA   
F1  2.607429e+01  5.899177e+01  3.702422e-01          40.0   65.0   30.0   
F2  6.099691e+01  1.272459e+02  9.300158e-02          64.0   94.0   75.0   
F3  6.620952e+01  1.514285e+02  2.510607e-01          68.0   98.0  105.0   
F4  1.041314e+02  2.523741e+02  5.819760e-01          80.0  118.0  140.0   
G1  7.648733e+01  2.205148e+02  4.402576e-01         135.2  200.0   60.0   
G2  4.102747e+01  3.039920e+02  7.887457e-01         158.0  248.0   75.0   
G3  1.036665e+02  5.481647e+02  1.604839e+00         472.2  563.4  120.0   
G4  7.594769e+01  5.571472e+02  1.679537e+00         492.8  611.8  140.0   
K1  3.862822e+09  2.052043e+10  1.100000e+08          84.4  139.4   40.0   
K2  1.178097e+02  7.639946e+02  1.073654e+00         147.4  201.0   90.0   
K3  1.514613e+02  7.884370e+02  5.076260e+00         182.4  242.0  120.0   
K4  2.814559e+02  3.687773e+03  6.400000e+08         212.2  270.4  135.0   

            
   Choco-Q  
F1    44.0  
F2   377.0  
F3  1337.0  
F4  1995.0  
G1   167.3  
G2   369.6  
G3   595.2  
G4  1640.4  
K1   114.8  
K2   385.5  
K3   536.5  
K4   613.9

The results in this table may not completely align with those in Table II due to the random generation of benchmark configurations. Nevertheless, it is evident that Choco-Q demonstrates a significant advantage over other baselines. In the following, we calculate the improvement over Cyclic, the state-of-art baseline.

## Caculate the improvement over Cyclic

In [3]:
# Assuming 'merged_df' already contains the necessary data (after the previous steps)
# Calculate the improvement for each row

# Circuit depth improvement: cyclic / Choco-Q
merged_df['Circuit_depth_improvement'] = merged_df[('Circuit depth', 'Cyclic')] / merged_df[('Circuit depth', 'Choco-Q')]

# Success rate improvement: Choco-Q / cyclic
merged_df['Success_rate_improvement'] = merged_df[('Success rate (%)', 'Choco-Q')] / merged_df[('Success rate (%)', 'Cyclic')]

# In-constraints rate improvement: Choco-Q / cyclic
merged_df['In_constraints_rate_improvement'] = merged_df[('In-constraints rate (%)', 'Choco-Q')] / merged_df[('In-constraints rate (%)', 'Cyclic')]

# Filter out rows where any improvement column has a zero denominator or zero numerator (to avoid division by zero)
valid_rows = merged_df[(merged_df[('Circuit depth', 'Cyclic')] != 0) & (merged_df[('Circuit depth', 'Choco-Q')] != 0) &
                       (merged_df[('Success rate (%)', 'Cyclic')] != 0) & (merged_df[('Success rate (%)', 'Choco-Q')] != 0) &
                       (merged_df[('In-constraints rate (%)', 'Cyclic')] != 0) & (merged_df[('In-constraints rate (%)', 'Choco-Q')] != 0)]

# Calculate the average improvement for each metric
avg_circuit_depth_improvement = valid_rows['Circuit_depth_improvement'].mean()
avg_success_rate_improvement = valid_rows['Success_rate_improvement'].mean()
avg_in_constraints_rate_improvement = valid_rows['In_constraints_rate_improvement'].mean()

improvement_table = pd.DataFrame({
    'Circuit Depth': [avg_circuit_depth_improvement],
    'Success Rate': [avg_success_rate_improvement],
    'In-constraints Rate': [avg_in_constraints_rate_improvement]
}, index=['Improvement relative to Cyclic'])
improvement_table

,Circuit Depth,Success Rate,In-constraints Rate
Improvement relative to Cyclic,0.811083,351.955239,53.95638
